In [1]:
import torch
from tqdm import tqdm, tqdm_notebook
from torchvision.datasets import MNIST
import torchvision.transforms as tt
from torch import nn
from torch.utils.data import DataLoader

In [2]:
transformation = tt.Compose([
    tt.ToTensor(),])
train_data = MNIST(root='F:/РАБОТА/diplom_reforked/',download=True,transform=transformation)
test_data = MNIST(root='F:/РАБОТА/diplom_reforked/',download=True,train=False,transform=transformation)
batch_size = 64

In [3]:
train_dataloader = DataLoader(train_data,batch_size = batch_size,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size = batch_size,shuffle=True)

In [4]:
import torch

class classic_lenet(nn.Module):
    def __init__(self):
        super(classic_lenet,self).__init__()
        self.conv1 = nn.Sequential(*[nn.Conv2d(1,32,kernel_size=3),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2)])
        self.conv2 = nn.Sequential(*[nn.Conv2d(32,64,kernel_size=3),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2)])
        self.conv3 = nn.Sequential(*[nn.Conv2d(64,64,kernel_size=3),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2)])
        self.flatten = nn.Flatten()
        self.fc1 = nn.Sequential(*[nn.Linear(1600,64),
                                   nn.ReLU()])
        self.fc2 = nn.Sequential(*[nn.Linear(64,10)])
        
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc1(self.flatten(x))
        x = self.fc2(x)
        return x
        
        

In [5]:
DEVICE = torch.device("cuda")


def train(train_loader, val_loader, model, opt, criterion=nn.CrossEntropyLoss(), epochs=30,
          path_to_save='best_classic_model.pth'):
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        val_max_loss = float('inf')
        patience = 3
        counter = 0
        best_accuracy = 0
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("train_loss", train_loss)
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            print("val_loss", val_loss)
            history.append((train_loss, train_acc, val_loss, val_acc))
#             if val_acc > best_accuracy:
#                 torch.save(model, path_to_save)
#                 best_accuracy = val_acc
            if val_loss < val_max_loss:
                counter = 0
                val_max_loss = val_loss
                torch.save(model, path_to_save)
            else:
                print(f'сработал счетчик на {epoch} эпохе')
                counter += 1
                if counter == patience:
                    return model
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch + 1, t_loss=train_loss, \
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
    return history


def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
        del loss
        del preds
        del outputs
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc


def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
        del inputs
        del labels
        del outputs
        del loss
        del preds
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

### Build Model

In [6]:
classic_model = classic_lenet()
optimizer = torch.optim.Adam(params=classic_model.parameters())
classic_model = classic_model.to(DEVICE)
loss = torch.nn.CrossEntropyLoss()

In [7]:
train(train_dataloader,test_dataloader,classic_model,optimizer,criterion=loss,epochs=10)

epoch:   0%|                                                                                    | 0/10 [00:00<?, ?it/s]

train_loss 0.2163699336429437


epoch:  10%|███████▌                                                                    | 1/10 [00:13<02:00, 13.42s/it]

val_loss 0.06379021574854851

Epoch 001 train_loss: 0.2164     val_loss 0.0638 train_acc 0.9355 val_acc 0.9800
train_loss 0.06263534424975514


epoch:  20%|███████████████▏                                                            | 2/10 [00:22<01:25, 10.63s/it]

...

epoch:  60%|█████████████████████████████████████████████▌                              | 6/10 [01:03<00:41, 10.50s/it]

val_loss 0.032386009434238076
сработал счетчик на 5 эпохе

Epoch 006 train_loss: 0.0224     val_loss 0.0324 train_acc 0.9930 val_acc 0.9897
train_loss 0.018494523535544673


epoch:  60%|█████████████████████████████████████████████▌                              | 6/10 [01:14<00:49, 12.41s/it]

val_loss 0.03548372002393007
сработал счетчик на 6 эпохе


classic_lenet(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Sequential(
    (0): Linear(in_features=1600, out_features=64, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)